In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
import time
import Load_Data_Patient
import pickle
from IPython.display import clear_output
from dateutil import parser
from datetime import datetime, time
import itertools
from tqdm import tqdm
import sys
import warnings
import json
import datetime
import Calculate_Time_Domain as td
import time
import dask.dataframe as dd
from dask.multiprocessing import get
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar
if not sys.warnoptions:
    warnings.simplefilter("ignore")
# pbar = tqdm(total=36)
# pbar.update(1)
# pbar.close()

# Calculate_File_Pure("C0161//22_11_2018_000000-22_11_2018_092830.annauxmix", 1, 2)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def Calculate_File_Pure(File, Interval, End): 
    global Failed
    try:
        File = "..//Patient_Data_Files//" + str(File)
        Segmented = Load_Data_Patient.Get_Segmented_Table(File, Interval, End)
        Segmented["Beats"] = 0
        Segmented["Beats"] = np.insert(np.diff(Segmented["RR Peaks"]), 0, 0)
        Segmented.drop([0,1], inplace=True)

        Test_Table = Segmented[["RR Peaks", "Beats", "Symbols"]]
        Test_Table.reset_index(drop=True, inplace=True)
        Test_Table["Mask"] = 1
        Test_Table["Reset"] = 0
        Test_Table["Percentage_Difference"] = 0

        Allowed_Symbols = ["N", "F", "+,(N", "+,(VT"]

        Test_Table["Index"] = Test_Table.index.values.tolist()
        Allowed_Symbols = ["N", "F"]

        def Symbol_Checker_Mask(Index):

            Current = Test_Table.loc[Index, "Symbols"]

            if Current in Allowed_Symbols:
                Test_Table.loc[Index,"Mask"] = 1
            else:
                Test_Table.loc[Index,"Mask"] = 0
                try:
                    if Test_Table.loc[Index+1,"Symbols"] in Allowed_Symbols:
                        Test_Table.loc[Index+2,"Reset"] = 1
                    else:
                        pass 
                except:
                    pass

        Test_Table["Index"].apply(Symbol_Checker_Mask)

        def Percentage_Generator_Column(Index):
            nonlocal Base
            try:
                To_Compare = Test_Table.loc[Index,"Beats"]
                Percentage_Difference = abs(((Base-To_Compare)/Base)*100)
                Test_Table.loc[Index,"Percentage_Difference"] = Percentage_Difference
                Base = Test_Table.loc[Index,"Beats"]
            except:
                pass

        Base = Test_Table["Beats"][1]
        Test_Table["Index"].apply(Percentage_Generator_Column)

        def Previous_Symbol_Checker(Index):

            Current_Symbols = Test_Table.loc[Index]["Symbols"]

            try:
                Previous_Symbols = Test_Table.loc[Index-1]["Symbols"]
                if Previous_Symbols in Allowed_Symbols:
                    pass

                elif Current_Symbols=="N" and Previous_Symbols=="V":
                    Test_Table.loc[Index,"Mask"] = 1

                else:
                    Test_Table.loc[Index,"Mask"] = 0
            except:
                pass


        Test_Table["Index"].apply(Previous_Symbol_Checker)

        def Isolated_Elemets_Percentage(Index):

            try:
                Previous = Test_Table.loc[Index-1, "Percentage_Difference"]
                Current = Test_Table.loc[Index, "Percentage_Difference"]
                Future = Test_Table.loc[Index+1, "Percentage_Difference"]

                if Previous >= 15 and Future >= 15:
                    if Current <=15:
                        Test_Table.loc[Index, "Mask"] = 0
                    else:
                        pass
                elif Previous <= 15 and Future <= 15:
                    if Current >=15:
                        Test_Table.loc[Index, "Mask"] = 0
                    else:
                        pass
            except:
                pass

        Test_Table["Index"].apply(Isolated_Elemets_Percentage)

        Indexes = np.array(Test_Table.loc[Test_Table["Symbols"] == "+,(SIGNALLOST"].index)

        for i in range(Indexes.shape[0]):
            try:

                Current = int(Indexes[i])
                Future = int(Indexes[i+1])
                Sum = np.sum(Test_Table.loc[range(Current, Future), "Beats"])
                if Sum <= 2:
                    Test_Table.drop(range(Current,Future), inplace=True)
                    del Current
                    del Future
                    del Sum
                else:
                    del Current
                    del Future
                    del Sum
            except:
                pass

        Test_Table.reset_index(drop=True, inplace=True)
        Test_Table["Index"] = Test_Table.index.values.tolist()

        def Signal_Lost_Elements(Index):

            try:
                if Test_Table.loc[Index, "Symbols"] == "+,(SIGNALLOST":
                    Test_Table.loc[Index+2, "Mask"] = 0
                else:
                    pass
            except:
                pass

        Test_Table["Index"].apply(Signal_Lost_Elements)
        Test_Table["Pure"] = 0

        def Isolated_Elemets_Mask(Index):

            try:
                Previous = Test_Table.loc[Index-1, "Mask"]
                Current = Test_Table.loc[Index, "Mask"]
                Future = Test_Table.loc[Index+1, "Mask"]

                if Previous == 0 and Future == 0:
                    Test_Table.loc[Index, "Mask"] = 0
            except:
                pass

        Test_Table["Index"].apply(Isolated_Elemets_Mask)

        Pure = Load_Data_Patient.Get_Pure_NNs(Test_Table)
        Pure.dropna(inplace=True)
        del Test_Table

        def Extremes(Index):

            if Pure.loc[Index, "Beats"] <= 53:
                Pure.loc[Index, "Pure"] = 0
            elif Pure.loc[Index, "Beats"] >= 153:
                Pure.loc[Index, "Pure"] = 0
            else:
                Pure.loc[Index, "Pure"] = 1

        Pure["Index"].apply(Extremes)

        Pure["Percentage"] = 0

        Base = Pure["Beats"][0]

        def Percentage_Generator_Column(Index):
            nonlocal Base
            if Pure.loc[Index, "Pure"] == 1:
                try:
                    To_Compare = Pure.loc[Index,"Beats"]
                    Percentage_Difference = int(round(abs(((Base-To_Compare)/Base)*100)))
                    Pure.loc[Index,"Percentage"] = Percentage_Difference
                    Base = Pure.loc[Index,"Beats"]
                except:
                    Base = float(Pure.loc[Index,"Beats"])


        Pure["Index"].apply(Percentage_Generator_Column)

        Pure["Peglano"] = 0 
        def Peglanje(Index):
            if Pure.loc[Index, "Pure"] == 1:
                try:
                    if Pure.loc[Index, "Percentage"] > 5:
                        PreviousAVG = np.average([Pure.loc[Index-2, "Beats"], Pure.loc[Index-1, "Beats"]])
                        CurrentAVG = np.average([Pure.loc[Index, "Beats"], Pure.loc[Index+1, "Beats"]])
                        FutureAVG = np.average([Pure.loc[Index+2, "Beats"], Pure.loc[Index+3, "Beats"]])

                        Percentage_Difference_Previous = int(round(abs(((CurrentAVG-PreviousAVG)/PreviousAVG)*100)))
                        Percentage_Difference_Future = int(round(abs(((CurrentAVG-FutureAVG)/FutureAVG)*100)))

                        if Percentage_Difference_Previous <= 2 and Percentage_Difference_Future <= 2:
        #                     Peglano.append(Pure.loc[Index, "Beats"])
                            Pure.loc[Index, "Peglano"] = 1
                        else:
                            pass
                    else:
        #                 Peglano.append(Pure.loc[Index, "Beats"])
                        Pure.loc[Index, "Peglano"] = 1
                except:
                    pass
            else:
                pass

        Peglano = []
        Pure["Index"].apply(Peglanje)

        def Isolated_Elemets_Mask(Index):
                if Pure.loc[Index, "Peglano"] == 1:
                    try:
                        Previous = Pure.loc[Index-1, "Peglano"]
                        Current = Pure.loc[Index, "Peglano"]
                        Future = Pure.loc[Index+1, "Peglano"]
                        if Previous == 0 and Future == 0:
                            Pure.loc[Index, "Peglano"] = 0
                        else:
                            pass
                    except:
                        pass

        Pure["Index"].apply(Isolated_Elemets_Mask)

        Temp_Second_Total = []
        Temp_Indexes = []
        for Row in Pure.iterrows():
            if Row[1][-1] == 0:
                if len(Temp_Second_Total) != 0:
                    Temp_Second_Total = np.sum(Temp_Second_Total)
                    if Temp_Second_Total >= 5:
                        Temp_Second_Total = []
                        Temp_Indexes = []
                    else:
                        Pure.loc[Temp_Indexes, "Peglano"] = 0
                        Temp_Second_Total = []
                        Temp_Indexes = []
                else:
                    Temp_Second_Total = []
                    Temp_Indexes = []
            else:
                Temp_Second_Total.append(Row[1][0])
                Temp_Indexes.append(Row[0])
        del Temp_Indexes
        del Temp_Second_Total

        Peglano = np.array(Pure.loc[Pure["Peglano"] == 1]["Beats"])

        if len(Peglano) >= 100:
            Results = Load_Data_Patient.Calculate_Pure_Td(Peglano)
            f= open("1h_Interval_Results//" + str(Interval) + "h_"+ str(End) +"h_AllPatientsParralel.txt","a+")
            To_Write = str(File).split("//")[-2] + "|" + str(Interval) + "|" + str(End) + "|" + str(File).split("//")[-1] + "|" + str(Results["Pure_SDNN"]/128) + "|" + str(Results["Pure_rMSSD"]/128) + "\n"
            f.write(To_Write)
            f.close()
            del Results
        else:
            Results = Load_Data_Patient.Calculate_Pure_Td(Peglano)
            f= open("1h_Interval_Results//" + str(Interval) + "h_"+ str(End) +"h_AllPatientsParralel.txt","a+")
            To_Write = str(File).split("//")[-2] + "|" + str(Interval) + "|" + str(End) + "|" + str(File).split("//")[-1] + "|" + str("Less than 100 beats") + "|" + str("Less than 100 beats") + "\n"
            f.write(To_Write)
            f.close()
            del Results
    except:
        Failed.append(File, Interval, End)

In [3]:
All_Files_Table = pd.read_excel("../Make_Input_Table/Valids.xlsx")

In [4]:
All_Files_Table["Baseline"].unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24,  1], dtype=int64)

In [5]:
Two = All_Files_Table.loc[All_Files_Table["Before"] == 2]
Two.head()

,Patient_ID,File,Before,Baseline,Valid,Index
3,C0161,C0161//21_11_2018_204830-22_11_2018_000000.ann...,2,3,True,3
4,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,2,3,True,4
8,C0161,C0161//21_11_2018_204830-22_11_2018_000000.ann...,2,4,True,8
9,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,2,4,True,9
12,C0161,C0161//21_11_2018_204830-22_11_2018_000000.ann...,2,5,True,12


In [6]:
Two.shape[0]

5489

In [7]:
ddata = dd.from_pandas(Two, npartitions=30)

In [ ]:
client = Client()

In [ ]:
client

In [8]:
Failed = []
Apply = ddata.apply(lambda X: Calculate_File_Pure(X['File'],
                                                  X["Before"],
                                                  X["Baseline"]),
                                                  axis=1,
                                                  meta=(object))
# 10 patitions That took 537.7677550315857 seconds 1@14 150
# 30 partitiona That took 482.91249990463257 seconds 1@5 250 / 121p 501s 1@5 250r
# 100 partiotions That took 488.9536120891571 seconds 1@6 184
# That took 9420.142078876495 seconds
# That took 16390.28028845787 seconds
starttime = time.time()
# with ProgressBar():
Apply.compute()
print('That took {} seconds'.format(time.time() - starttime))
# pbar.close()

8495it [22:21:50, 14.32s/it]

That took 80510.08154129982 seconds
